# Feature Engineering and Modelling

## Outline

1. Imports
2. Feature Engineering
    a. Price Features
    b. Date features
3. Model Training and Hyperparameter Tuning
4. Model Evaluation and Prediction

In [1]:
#imports

import pandas as pd
from functools import reduce

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)

pd.set_option('display.max_columns', None)

/opt/miniconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#load data
client_date_cols = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
client_data = pd.read_csv("../data/raw/client_data.csv", parse_dates = client_date_cols)
price_data = pd.read_csv("../data/raw/price_data.csv", parse_dates = ['price_date'])

### Preprocessing

In [3]:
client_data.shape

(14606, 26)

In [4]:
client_data['channel_sales'].value_counts()

foosdfpfkusacimwkcsosbicdxkicaua    6754
MISSING                             3725
lmkebamcaaclubfxadlmueccxoimlema    1843
usilxuppasemubllopkaafesmlibmsdf    1375
ewpakwlliwisiwduibdlfmalxowmwpci     893
sddiedcslfslkckwlfkdpoeeailfpeds      11
epumfxlbckeskwekxbiuasklxalciiuu       3
fixdbufsefwooaasfcxdxadsiekoceaa       2
Name: channel_sales, dtype: int64

In [5]:
client_data['origin_up'].value_counts()

lxidpiddsbxsbosboudacockeimpuepw    7097
kamkkxfxxuwbdslkwifmmcsiusiuosws    4294
ldkssxwpmemidmecebumciepifcamkci    3148
MISSING                               64
usapbepcfoloekilkwsdiboslwaxobdp       2
ewxeelcelemmiwuafmddpobolfuxioce       1
Name: origin_up, dtype: int64

We can see that there are channel_sales and origin_up values that are only associated with very few records. We will drop these rows. 

In [6]:
channel_sales_vals = ['epumfxlbckeskwekxbiuasklxalciiuu', 'fixdbufsefwooaasfcxdxadsiekoceaa']
origin_up_vals = ['usapbepcfoloekilkwsdiboslwaxobdp', 'ewxeelcelemmiwuafmddpobolfuxioce']

clean_client_data = client_data[client_data.channel_sales.isin(channel_sales_vals) == False]
clean_client_data = client_data[client_data.origin_up.isin(origin_up_vals) == False]

In [7]:
clean_client_data.shape

(14603, 26)

In [8]:
# clean_client_data.to_csv('../data/processed/clean_client_data.csv')

### Feature engineering

#### 1. Price Features

Approach 1: Add features of price differences between December and the preceding January for off-peak, mid-peak and peak prices. 

In [9]:
## off-peak
mon_price_by_com = price_data.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean',
                                                                 'price_off_peak_fix': 'mean'}).reset_index()
jan_prices = mon_price_by_com.groupby('id').first().reset_index()
dec_prices = mon_price_by_com.groupby('id').last().reset_index()

jan_prices = jan_prices.rename(columns = {'price_off_peak_var': 'eng_jan', 'price_off_peak_fix': 'pow_jan'})
dec_prices = dec_prices.rename(columns = {'price_off_peak_var': 'eng_dec', 'price_off_peak_fix': 'pow_dec'})
diff_off = pd.merge(jan_prices, dec_prices.drop(columns = 'price_date'), on = 'id')
diff_off['offpeak_dec_jan_diff_eng'] = diff_off['eng_dec'] - diff_off['eng_jan']
diff_off['offpeak_dec_jan_diff_pow'] = diff_off['pow_dec'] - diff_off['pow_jan']
diff_off = diff_off[['id', 'offpeak_dec_jan_diff_eng', 'offpeak_dec_jan_diff_pow']]
diff_off.head()

,id,offpeak_dec_jan_diff_eng,offpeak_dec_jan_diff_pow
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [10]:
## mid
mon_price_by_com_mid = price_data.groupby(['id', 'price_date']).agg({'price_mid_peak_var': 'mean',
                                                                 'price_mid_peak_fix': 'mean'}).reset_index()

jan_prices = mon_price_by_com_mid.groupby('id').first().reset_index()
dec_prices = mon_price_by_com_mid.groupby('id').last().reset_index()

jan_prices = jan_prices.rename(columns = {'price_mid_peak_var': 'eng_jan', 'price_mid_peak_fix': 'pow_jan'})
dec_prices = dec_prices.rename(columns = {'price_mid_peak_var': 'eng_dec', 'price_mid_peak_fix': 'pow_dec'})
diff_mid = pd.merge(jan_prices, dec_prices.drop(columns = 'price_date'), on = 'id')
diff_mid['midpeak_dec_jan_diff_eng'] = diff_mid['eng_dec'] - diff_mid['eng_jan']
diff_mid['midpeak_dec_jan_diff_pow'] = diff_mid['pow_dec'] - diff_mid['pow_jan']
diff_mid = diff_mid[['id', 'midpeak_dec_jan_diff_eng', 'midpeak_dec_jan_diff_pow']]
diff_mid.head()

,id,midpeak_dec_jan_diff_eng,midpeak_dec_jan_diff_pow
0,0002203ffbb812588b632b9e628cc38d,0.003487,0.065166
1,0004351ebdd665e6ee664792efc4fd13,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,0.000763,0.065166
4,00114d74e963e47177db89bc70108537,0.000000,0.000000


In [11]:
## peak
mon_price_by_com_peak = price_data.groupby(['id', 'price_date']).agg({'price_peak_var': 'mean',
                                                                 'price_peak_fix': 'mean'}).reset_index()
jan_prices = mon_price_by_com_peak.groupby('id').first().reset_index()
dec_prices = mon_price_by_com_peak.groupby('id').last().reset_index()

jan_prices = jan_prices.rename(columns = {'price_peak_var': 'eng_jan', 'price_peak_fix': 'pow_jan'})
dec_prices = dec_prices.rename(columns = {'price_peak_var': 'eng_dec', 'price_peak_fix': 'pow_dec'})
diff_peak = pd.merge(jan_prices, dec_prices.drop(columns = 'price_date'), on = 'id')
diff_peak['peak_dec_jan_diff_eng'] = diff_peak['eng_dec'] - diff_peak['eng_jan']
diff_peak['peak_dec_jan_diff_pow'] = diff_peak['pow_dec'] - diff_peak['pow_jan']
diff_peak = diff_peak[['id', 'peak_dec_jan_diff_eng', 'peak_dec_jan_diff_pow']]
diff_peak.head()

,id,peak_dec_jan_diff_eng,peak_dec_jan_diff_pow
0,0002203ffbb812588b632b9e628cc38d,-0.002302,0.097749
1,0004351ebdd665e6ee664792efc4fd13,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,-0.005120,0.097749
4,00114d74e963e47177db89bc70108537,0.000000,0.000000


In [12]:
#merge dfs
diff_dfs = [diff_off, diff_mid, diff_peak]
diff = reduce(lambda left, right: pd.merge(left, right, on='id'), diff_dfs)
diff.head()

,id,offpeak_dec_jan_diff_eng,offpeak_dec_jan_diff_pow,midpeak_dec_jan_diff_eng,midpeak_dec_jan_diff_pow,peak_dec_jan_diff_eng,peak_dec_jan_diff_pow
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916,0.003487,0.065166,-0.002302,0.097749
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779,0.000000,0.000000,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000,0.000000,0.000000,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916,0.000763,0.065166,-0.005120,0.097749
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001,0.000000,0.000000,0.000000,0.000000


In [13]:
join_data_1 = pd.merge(diff, clean_client_data, on = 'id')
join_data_1.head()

,id,offpeak_dec_jan_diff_eng,offpeak_dec_jan_diff_pow,midpeak_dec_jan_diff_eng,midpeak_dec_jan_diff_pow,peak_dec_jan_diff_eng,peak_dec_jan_diff_pow,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916,0.003487,0.065166,-0.002302,0.097749,foosdfpfkusacimwkcsosbicdxkicaua,22034,0,3084,2010-01-19,2016-02-21,2010-01-19,2015-02-25,729.06,425,0.0,138.95,0.116900,0.100015,40.606701,f,40.78,43.08,43.08,1,81.42,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,17.250,0
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779,0.000000,0.000000,0.000000,0.000000,MISSING,4060,0,0,2009-08-06,2016-06-21,2013-06-21,2015-06-23,597.77,0,0.0,6.84,0.142065,0.000000,44.311378,f,0.00,24.42,24.42,1,61.58,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000,0.000000,0.000000,0.000000,0.000000,usilxuppasemubllopkaafesmlibmsdf,7440,0,1062,2013-02-25,2016-05-05,2015-05-05,2015-02-26,1311.16,1062,30.0,18.37,0.199230,0.000000,45.806878,f,213.76,38.58,38.58,2,81.61,3,lxidpiddsbxsbosboudacockeimpuepw,13.856,0
3,00114d74e963e47177db89bc70108537,-0.003994,-0.000001,0.000000,0.000000,0.000000,0.000000,ewpakwlliwisiwduibdlfmalxowmwpci,11272,0,0,2010-03-02,2016-03-02,2010-03-02,2015-03-09,1671.41,0,0.0,18.27,0.144149,0.000000,44.311378,f,0.00,29.76,29.76,1,157.99,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
4,0013f326a839a2f6ad87a1859952d227,-0.006171,0.000000,0.003371,0.000000,-0.002351,0.000000,foosdfpfkusacimwkcsosbicdxkicaua,267414,0,19394,2013-02-22,2016-02-22,2014-10-07,2015-02-26,3077.34,1760,0.0,144.86,0.118636,0.101750,40.606701,f,195.20,30.00,30.00,1,341.58,3,lxidpiddsbxsbosboudacockeimpuepw,20.000,0


Approach 2: Add features of price differences between consecutive periods for off-peak, mid-peak and peak prices. 

In [14]:
# Aggregate average prices per period by company
mean_prices = price_data.groupby(['id']).agg({
    'price_off_peak_var': 'mean', 
    'price_peak_var': 'mean', 
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean'    
}).reset_index()

In [15]:
# Calculate the mean difference between periods
mean_prices['off_peak_peak_var_mean_diff'] = mean_prices['price_off_peak_var'] - mean_prices['price_peak_var']
mean_prices['peak_mid_peak_var_mean_diff'] = mean_prices['price_peak_var'] - mean_prices['price_mid_peak_var']
mean_prices['off_peak_mid_peak_var_mean_diff'] = mean_prices['price_off_peak_var'] - mean_prices['price_mid_peak_var']
mean_prices['off_peak_peak_fix_mean_diff'] = mean_prices['price_off_peak_fix'] - mean_prices['price_peak_fix']
mean_prices['peak_mid_peak_fix_mean_diff'] = mean_prices['price_peak_fix'] - mean_prices['price_mid_peak_fix']
mean_prices['off_peak_mid_peak_fix_mean_diff'] = mean_prices['price_off_peak_fix'] - mean_prices['price_mid_peak_fix']

In [16]:
mean_prices.head()

,id,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,off_peak_peak_var_mean_diff,peak_mid_peak_var_mean_diff,off_peak_mid_peak_var_mean_diff,off_peak_peak_fix_mean_diff,peak_mid_peak_fix_mean_diff,off_peak_mid_peak_fix_mean_diff
0,0002203ffbb812588b632b9e628cc38d,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,0.020545,0.030633,0.051178,16.280694,8.140345,24.421038
1,0004351ebdd665e6ee664792efc4fd13,0.146426,0.000000,0.000000,44.385450,0.000000,0.000000,0.146426,0.000000,0.146426,44.385450,0.000000,44.385450
2,0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.000000,0.000000,45.319710,0.000000,0.000000,0.181558,0.000000,0.181558,45.319710,0.000000,45.319710
3,0010ee3855fdea87602a5b7aba8e42de,0.118757,0.098292,0.069032,40.647427,24.388455,16.258971,0.020465,0.029260,0.049725,16.258972,8.129484,24.388456
4,00114d74e963e47177db89bc70108537,0.147926,0.000000,0.000000,44.266930,0.000000,0.000000,0.147926,0.000000,0.147926,44.266930,0.000000,44.266930


In [17]:
diff_cols = [col for col in mean_prices.columns if 'diff' in col]
columns = diff_cols + ['id']
join_data_2 = pd.merge(clean_client_data, mean_prices[columns], on='id')
join_data_2.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn,off_peak_peak_var_mean_diff,peak_mid_peak_var_mean_diff,off_peak_mid_peak_var_mean_diff,off_peak_peak_fix_mean_diff,peak_mid_peak_fix_mean_diff,off_peak_mid_peak_fix_mean_diff
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1,0.024038,0.034219,0.058257,18.590255,7.450670,26.040925
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0,0.142485,0.007124,0.149609,44.311375,0.000000,44.311375
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0,0.082090,0.088421,0.170512,44.385450,0.000000,44.385450
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0,0.151210,0.000000,0.151210,44.400265,0.000000,44.400265
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0,0.020536,0.030773,0.051309,16.275263,8.137629,24.412893


Approach 3: Add features of max price differences between consecutive periods for off-peak, mid-peak and peak prices across different months. 

In [18]:
# Aggregate average prices per period by company
mean_prices_by_month = price_data.groupby(['id', 'price_date']).agg({
    'price_off_peak_var': 'mean', 
    'price_peak_var': 'mean', 
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean'    
}).reset_index()

In [19]:
# Calculate the mean difference between consecutive periods
mean_prices_by_month['off_peak_peak_var_mean_diff'] = mean_prices_by_month['price_off_peak_var'] - mean_prices_by_month['price_peak_var']
mean_prices_by_month['peak_mid_peak_var_mean_diff'] = mean_prices_by_month['price_peak_var'] - mean_prices_by_month['price_mid_peak_var']
mean_prices_by_month['off_peak_mid_peak_var_mean_diff'] = mean_prices_by_month['price_off_peak_var'] - mean_prices_by_month['price_mid_peak_var']
mean_prices_by_month['off_peak_peak_fix_mean_diff'] = mean_prices_by_month['price_off_peak_fix'] - mean_prices_by_month['price_peak_fix']
mean_prices_by_month['peak_mid_peak_fix_mean_diff'] = mean_prices_by_month['price_peak_fix'] - mean_prices_by_month['price_mid_peak_fix']
mean_prices_by_month['off_peak_mid_peak_fix_mean_diff'] = mean_prices_by_month['price_off_peak_fix'] - mean_prices_by_month['price_mid_peak_fix']

In [20]:
# Calculate the maximum monthly difference across time periods
max_diff_across_periods_months = mean_prices_by_month.groupby(['id']).agg({
    'off_peak_peak_var_mean_diff': 'max',
    'peak_mid_peak_var_mean_diff': 'max',
    'off_peak_mid_peak_var_mean_diff': 'max',
    'off_peak_peak_fix_mean_diff': 'max',
    'peak_mid_peak_fix_mean_diff': 'max',
    'off_peak_mid_peak_fix_mean_diff': 'max'
}).reset_index().rename(
    columns={
        'off_peak_peak_var_mean_diff': 'off_peak_peak_var_max_monthly_diff',
        'peak_mid_peak_var_mean_diff': 'peak_mid_peak_var_max_monthly_diff',
        'off_peak_mid_peak_var_mean_diff': 'off_peak_mid_peak_var_max_monthly_diff',
        'off_peak_peak_fix_mean_diff': 'off_peak_peak_fix_max_monthly_diff',
        'peak_mid_peak_fix_mean_diff': 'peak_mid_peak_fix_max_monthly_diff',
        'off_peak_mid_peak_fix_mean_diff': 'off_peak_mid_peak_fix_max_monthly_diff'
    }
)

In [21]:
max_diff_across_periods_months.head()

,id,off_peak_peak_var_max_monthly_diff,peak_mid_peak_var_max_monthly_diff,off_peak_mid_peak_var_max_monthly_diff,off_peak_peak_fix_max_monthly_diff,peak_mid_peak_fix_max_monthly_diff,off_peak_mid_peak_fix_max_monthly_diff
0,0002203ffbb812588b632b9e628cc38d,0.022225,0.033743,0.055866,16.291555,8.145775,24.437330
1,0004351ebdd665e6ee664792efc4fd13,0.148405,0.000000,0.148405,44.444710,0.000000,44.444710
2,0010bcc39e42b3c2131ed2ce55246e3c,0.205742,0.000000,0.205742,45.944710,0.000000,45.944710
3,0010ee3855fdea87602a5b7aba8e42de,0.022581,0.031859,0.054440,16.291555,8.145775,24.437330
4,00114d74e963e47177db89bc70108537,0.149902,0.000000,0.149902,44.266931,0.000000,44.266931


In [22]:
join_data_3 = pd.merge(clean_client_data, max_diff_across_periods_months, on='id')
join_data_3.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn,off_peak_peak_var_max_monthly_diff,peak_mid_peak_var_max_monthly_diff,off_peak_mid_peak_var_max_monthly_diff,off_peak_peak_fix_max_monthly_diff,peak_mid_peak_fix_max_monthly_diff,off_peak_mid_peak_fix_max_monthly_diff
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1,0.060550,0.085483,0.146033,44.266930,8.145775,44.26693
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0,0.151367,0.085483,0.151367,44.444710,0.000000,44.44471
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0,0.084587,0.089162,0.172468,44.444710,0.000000,44.44471
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0,0.153133,0.000000,0.153133,44.444710,0.000000,44.44471
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0,0.022225,0.033743,0.055866,16.291555,8.145775,24.43733


In [23]:
#define scoring function
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and standard deviation from cross validation.

    Parameters
    ----------
    model :
        model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data
    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)
    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []
    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))
    return pd.Series(data=out_col, index=mean_scores.index)

In [24]:
#define training function
def train_xgb(df, model_name):
    #split training and test data
    train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)
    target = 'churn'
    X_train, y_train = train_df.drop(columns = [target]), train_df[target]
    X_test, y_test = test_df.drop(columns = [target]), train_df[target]

    #set columns
    all_cols = df.columns.tolist()
    target = ['churn']

    date_feats = [col for col in all_cols if col.startswith('date')]
    forecast_feats = [col for col in all_cols if col.startswith('forecast')]
    categorical_feats = ['channel_sales', 'origin_up' ]
    binary_feats = ['has_gas']
    drop_feats = ['id']
    numerical_feats = list(set(all_cols) -
                       set(date_feats) -
                       set(categorical_feats) -
                       set(binary_feats) -
                       set(target)-
                       set(drop_feats))
    #preprocessor
    preprocessor = make_column_transformer(
        (OneHotEncoder(drop ='if_binary'), binary_feats),
        (OneHotEncoder(handle_unknown = 'ignore', sparse = False), categorical_feats),
        ("passthrough", numerical_feats)
        )
    #pipeline with xgb
    xgb = XGBClassifier(random_state = 42, verbosity = 0, use_label_encoder = False)
    pipe_xgb = make_pipeline(preprocessor, xgb)
    #metrics
    scoring_metric = ['accuracy', 'precision', 'recall', 'f1']
    results[model_name] = mean_std_cross_val_scores(pipe_xgb, X_train, y_train, return_train_score = True, scoring = scoring_metric)
    results_df = pd.DataFrame(results).T
    return results_df

In [25]:
results = {}

train_xgb(join_data_1, 'price difference between dec and jan')
train_xgb(join_data_2, 'price difference between periods')
train_xgb(join_data_3, 'max price difference between periods')

,fit_time,score_time,test_accuracy,train_accuracy,test_precision,train_precision,test_recall,train_recall,test_f1,train_f1
price difference between dec and jan,0.774 (+/- 0.016),0.009 (+/- 0.001),0.905 (+/- 0.001),0.973 (+/- 0.001),0.519 (+/- 0.025),0.999 (+/- 0.001),0.073 (+/- 0.016),0.714 (+/- 0.015),0.128 (+/- 0.024),0.833 (+/- 0.010)
price difference between periods,0.767 (+/- 0.010),0.009 (+/- 0.001),0.905 (+/- 0.002),0.970 (+/- 0.001),0.593 (+/- 0.072),0.999 (+/- 0.001),0.088 (+/- 0.010),0.697 (+/- 0.012),0.153 (+/- 0.018),0.821 (+/- 0.008)
max price difference between periods,0.813 (+/- 0.010),0.009 (+/- 0.000),0.904 (+/- 0.003),0.969 (+/- 0.001),0.546 (+/- 0.094),1.000 (+/- 0.001),0.082 (+/- 0.016),0.679 (+/- 0.007),0.143 (+/- 0.027),0.809 (+/- 0.005)


From these results, we can see that the second group of features performs the best in terms of all metrics. Also, we observe the following:
1. Accuracy score is high but this does not tell the whole story in our case, we need to look at other metrics. 
2. Test recall and test F1 scores are both low, so we will need to continue with feature engineering and hyperparameter optimization in order to improve on these metrics. 
3. Our models overfit because the training scores are much higher than the test scores so we will need to correct for it. 

#### 2. Date Features

In the models above, we did not make use of the date features:
- date_activ = date of activation of the contract
- date_end = registered date of the end of the contract
- date_modif_prod = date of the last modification of the product
- date_renewal = date of the next contract renewal

In order to make use to these features, we will transform them into months to reference data ('January 1st, 2016')

In [26]:
from datetime import datetime
import numpy as np
reference_date = datetime(2016, 1, 1)

def convert_months(reference_date, df, column):
    time_diff_months = reference_date - df[column]
    months = (time_diff_months / np.timedelta64(1, 'M')).astype(int)
    return months

In [27]:
client_feat = clean_client_data.copy()
client_feat['months_activ'] = convert_months(reference_date, clean_client_data, 'date_activ')
client_feat['months_to_end'] = -convert_months(reference_date, clean_client_data, 'date_end')
client_feat['months_modif_prod'] = convert_months(reference_date, clean_client_data, 'date_modif_prod')
client_feat['months_renewal'] = convert_months(reference_date, clean_client_data, 'date_renewal')
client_feat.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn,months_activ,months_to_end,months_modif_prod,months_renewal
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1,30,5,2,6
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0,76,7,76,4
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0,68,3,68,8
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0,69,2,69,9
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0,71,2,71,9


In [ ]:
join_data_4 = pd.merge(client_feat, mean_prices[columns], on='id')
train_xgb(join_data_4, 'price difference between periods + date feats')

With the inclusion of the transformed date features, we are able to improve the recall and the F1 scores. 

* We will explore if a derived feature from the date columns: contract length can be meaningful feature. 

In [ ]:
client_feat_2 = client_feat.copy()
client_feat_2['contract_length'] = ((client_feat_2['date_end'] - client_feat_2['date_activ'])/ np.timedelta64(1, 'Y')).astype(int)

In [ ]:
client_feat_2.groupby(['contract_length']).agg({'churn': 'mean'}).sort_values(by='churn', ascending=False)

We can see that clients with contract lengths of 2 - 4 years are most likely to churn. 
Clients with contract lengths of 8 - 10 years are least likely to churn. Interesting, clients clients with contracts for more than 10 years show various degrees of churning tendency.

I experimented with this feature, but it did not improve the model results, so we will not include it in the model. 